In [ ]:
!pip install rasterio google-cloud-storage -q
# If running from orchestrator

import rasterio
import numpy as np
from google.cloud import storage
from google.colab import auth
from io import BytesIO

# ============================================================
# 🔑 Authenticate & Initialize
# ============================================================
auth.authenticate_user()
cliente = storage.Client()
bucket_name = "nasa_sar_spacetron"
bucket = cliente.bucket(bucket_name)

# ============================================================
# 🗂️ Folders
# ============================================================
carpeta_entrada = "CLEAN/Interest"
carpeta_salida  = "CLEAN/Interest_clean"

# ============================================================
# 📋 List files in input & output
# ============================================================
blobs_in = list(cliente.list_blobs(bucket_name, prefix=carpeta_entrada))
archivos_entrada = [b for b in blobs_in if b.name.endswith(".tif")]

blobs_out = list(cliente.list_blobs(bucket_name, prefix=carpeta_salida))
archivos_salida = [b.name.split("/")[-1] for b in blobs_out]

# Filter out already cleaned ones
archivos_pendientes = [b for b in archivos_entrada if b.name.split("/")[-1] not in archivos_salida]

print(f"📦 Total archivos en origen: {len(archivos_entrada)}")
print(f"📦 Total archivos ya limpios: {len(archivos_salida)}")
print(f"🧹 Archivos pendientes por limpiar: {len(archivos_pendientes)}\n")

# ============================================================
# 🧽 Process only missing files
# ============================================================
for blob in archivos_pendientes:
    nombre = blob.name.split("/")[-1]
    print(f"🔄 Procesando: {nombre}")

    # Read from GCS
    contenido = blob.download_as_bytes()
    with rasterio.MemoryFile(contenido) as memfile:
        with memfile.open() as src:
            img = src.read(1)
            perfil = src.profile

    # Clean invalid data
    img = np.where(np.isfinite(img), img, np.nan)
    img = img.astype(np.float32)
    perfil.update(dtype=rasterio.float32, nodata=np.nan)

    # Write cleaned version to memory
    with rasterio.MemoryFile() as memfile_out:
        with memfile_out.open(**perfil) as dst:
            dst.write(img, 1)
        memfile_out.seek(0)
        blob_salida = bucket.blob(f"{carpeta_salida}/{nombre}")
        blob_salida.upload_from_file(memfile_out, rewind=True)

    print(f"✅ Guardado en: {carpeta_salida}/{nombre}\n")

print("🎯 Proceso completado. Solo se limpiaron archivos nuevos.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 52.6 MB/s eta 0:00:00
📦 Total archivos en origen: 94
📦 Total archivos ya limpios: 0
🧹 Archivos pendientes por limpiar: 94

🔄 Procesando: Interest.tif
✅ Guardado en: CLEAN/Interest_clean/Interest.tif

🔄 Procesando: AOI_1_2020_Q1.tif
✅ Guardado en: CLEAN/Interest_clean/AOI_1_2020_Q1.tif

🔄 Procesando: AOI_1_2020_Q2.tif
✅ Guardado en: CLEAN/Interest_clean/AOI_1_2020_Q2.tif

🔄 Procesando: AOI_1_2020_Q3.tif
✅ Guardado en: CLEAN/Interest_clean/AOI_1_2020_Q3.tif

🔄 Procesando: AOI_1_2020_Q4.tif
✅ Guardado en: CLEAN/Interest_clean/AOI_1_2020_Q4.tif

🔄 Procesando: AOI_1_2021_Q1.tif
✅ Guardado en: CLEAN/Interest_clean/AOI_1_2021_Q1.tif

🔄 Procesando: AOI_1_2021_Q2.tif
✅ Guardado en: CLEAN/Interest_clean/AOI_1_2021_Q2.tif

🔄 Procesando: AOI_1_2021_Q3.tif
✅ Guardado en: CLEAN/Interest_clean/AOI_1_2021_Q3.tif

🔄 Procesando: AOI_1_2021_Q4.tif
✅ Guardado en: CLEAN/Interest_clean/AOI_1_2021_Q4.tif

🔄 Procesando: AOI_1_2022_Q1.tif
✅ Guardado en: 